<div style="text-align: left;"><img src="https://www.juliabox.org/assets/img/juliacloudlogo.png" style="margin: 0px 0px 0px 0px; padding-right: 20px;width: 80px; float: left;" title="" alt="" /></div>
<img src="http://dmkpress.com/images/cms/thumbs/a5b0aeaa3fa7d6e58d75710c18673bd7ec6d5f6d/978-5-97060-370-3_270_369__100.jpg" style="margin: 0px 0px 5px 20px; width: 100px; float: right;" title="" alt="" />
Всестороннее введение в новый язык программирования для научно-технических вычислений [Julia](http://julialang.org/) в книге Малколма Шеррингтона, Packt Publishing, июль 2015.

<h1>Осваиваем язык Julia</h1><br />

Совершенствование мастерства в области аналитики и программирования при помощи Julia в целях решения задач комплексной обработки данных
<div style="text-align: left;font-size:8pt;padding-top:10px;">Программный код Julia (v0.4.5) протестирован в Windows 8.1/10 и Linux/Lubuntu 16.4</div>
<div style="text-align: left;"><h1>Глава 6. Примеры</h1></div>

## Стохастическое моделирование
### Пакет SimJulia
### Пример с клиентским обслуживанием в банке (вар. 2)

In [9]:
#
# bank_y.jl
#

using SimJulia
using Distributions

function customer(env::Environment, name::UTF8String, counter::Resource, time_in_bank::Float64)
  arrive = now(env)
    
  println("$arrive $name: прибыл")
    
  req = Request(counter)
  patience = rand(Uniform(MIN_PATIENCE, MAX_PATIENCE))
  result = yield(req | Timeout(env, patience))
    
  wait = now(env) - arrive
  push!(wait_times, wait);
    
  if in(req, keys(result))
    println("$(now(env)) $name: прождал $wait") 
    yield(Timeout(env, rand(Exponential(time_in_bank))))
    println("$(now(env)) $name: обслужен")    
    yield(Release(counter))
    #push!(in_bank_times, now(env) - arrive);
  else
    println("$(now(env)) $name: ОТКАЗАЛСЯ после $wait")
    push!(renege_qty, 1);       
    SimJulia.cancel(req)
  end
end

function source(env::Environment, number::Int, interval::Float64, counter::Resource)
  d = Exponential(interval)
  for i in 1:number
    Process(env, customer, "Клиент №$i", counter, 12.0)
    yield(Timeout(env, rand(d)))
  end
end

function randomize()  # lcg_rand
  seed = ccall( (:GetTickCount, "kernel32"), stdcall, Int32, () );
  ((seed * 214013 + 2531011) >> 16) & 0x7fff
end

# Настроить и запустить имитационный прогон
println("Отказы от услуги в банке\n")

# Установить исходные данные имитации 

const MAX_CUSTOMERS     = 20    # Общее число клиентов
const MAX_TIME          = 400.0 # Общее время работы банка
const MEAN_ARRIVAL_TIME = 10.0   # Генерировать новых клиентов с прим. промежутком x (интервалы)
const MIN_PATIENCE      = 2.0   # Минимальное терпение клиента
const MAX_PATIENCE      = 12.0   # Максимальное терпение клиента (среднее время обслуживания)
const MAX_TELLERS       = 1     # Число операционистов

# Использовать ранее определенную функцию randomize() либо 
# выбрать значение, чтобы сделать прогон воспроизводимым
#const RANDOM_SEED = 150  
const RANDOM_SEED = randomize()
srand(RANDOM_SEED)

global wait_times = Float64[];  # Времена ожидания
global renege_qty = Int32[];    # Количество отказов

env = Environment()

# Запустить процессы и выполнить
counters = Resource(env, MAX_TELLERS)
Process(env, source, MAX_CUSTOMERS, MEAN_ARRIVAL_TIME, counters)
run(env, MAX_TIME)

@printf "\nПри наличии %s операциониста(ов):\n" MAX_TELLERS
@printf "Среднее время ожидания = %5.3f\n" mean(wait_times)
@printf "Количество отказов = %5d" sum(renege_qty)

Отказы от услуги в банке

0.0 Клиент №1: прибыл
0.0 Клиент №1: прождал 0.0
1.769315966752774 Клиент №2: прибыл
2.3713121111379767 Клиент №3: прибыл
4.616666178605395 Клиент №3: ОТКАЗАЛСЯ после 2.245354067467418
5.653796774345105 Клиент №2: ОТКАЗАЛСЯ после 3.884480807592331
13.441481254776257 Клиент №4: прибыл
17.9312084894578 Клиент №4: ОТКАЗАЛСЯ после 4.489727234681542
18.5800650799264 Клиент №5: прибыл
23.5486727338523 Клиент №5: ОТКАЗАЛСЯ после 4.9686076539259005
34.72997874523776 Клиент №6: прибыл
34.835002671310285 Клиент №7: прибыл
39.54268711373291 Клиент №6: ОТКАЗАЛСЯ после 4.8127083684951515
39.63722791933332 Клиент №7: ОТКАЗАЛСЯ после 4.802225248023035
42.61805033927759 Клиент №1: обслужен
46.10565188302617 Клиент №8: прибыл
46.10565188302617 Клиент №8: прождал 0.0
50.5183890620335 Клиент №9: прибыл
53.573246425277574 Клиент №9: ОТКАЗАЛСЯ после 3.054857363244075
53.882967198943774 Клиент №8: обслужен
65.40476431411466 Клиент №10: прибыл
65.40476431411466 Клиент №10: прождал 0

Количество отказов =    11